In [1]:
### Asset Pricing - Assignment 3 

In [1]:
import pandas as pd
import random
import numpy as np
import pandas as pd
import pandas_datareader.data as web
import datetime
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import subplot, scatter, plot, axis
from scipy.stats import linregress
import ast 
import re
import seaborn as sns
import plotly.figure_factory as ff
import plotly.graph_objs as go
import scipy.stats as stats
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm

pd.set_option("display.notebook_repr_html", False)
pd.set_option("display.max_columns", 12)
pd.set_option("display.width", 80)
pd.set_option("precision", 3)

In [5]:
df_market_Portfolio = pd.read_csv("Market_Portfolio.csv")
data_indus_Portfolio  = pd.read_csv("Industry_Portfolios.csv")

df_risk_factors.head(10)
df_risk_factors.set_index("Date",inplace = True)
df_market_Portfolio.set_index("Date",inplace = True)
data_indus_Portfolio.set_index("Date",inplace = True)


In [9]:
df_ = pd.concat([data_indus_Portfolio,df_market_Portfolio],axis =1)

### part 1 : Estimated the expected deviation from market return for the ten industry portfolios

In [61]:
df_premium  = np.subtract( df_.loc[:,"NoDur":"Other"] , pd.DataFrame(df_["Market"]))
df_premium_mean  = df_premium.mean()
df_premium_mean

NoDur    0.155
Durbl   -0.015
Manuf    0.265
Enrgy    0.483
HiTec    0.018
Telcm    0.133
Shops    0.168
Hlth     0.036
Utils    0.159
Other   -0.259
dtype: float64

### Covariance matrix

In [60]:
Covmatrix = df_premium.cov()
Covmatrix

       NoDur   Durbl  Manuf   Enrgy  HiTec  Telcm  Shops   Hlth   Utils  Other
NoDur  5.440  -6.073 -1.396  -1.201 -1.883  1.539  1.141  3.815   4.272 -1.769
Durbl -6.073  26.629  4.908  -3.481  1.892 -1.708 -0.354 -8.083  -9.617  4.386
Manuf -1.396   4.908  2.950   1.666  0.065 -0.626 -1.155 -2.289  -1.901  0.359
Enrgy -1.201  -3.481  1.666  19.275 -1.517 -1.041 -3.710 -2.486   4.454 -3.865
HiTec -1.883   1.892  0.065  -1.517  5.099 -0.773 -0.245 -1.936  -2.343 -1.404
Telcm  1.539  -1.708 -0.626  -1.041 -0.773  4.683  0.464  0.693   2.721 -1.272
Shops  1.141  -0.354 -1.155  -3.710 -0.245  0.464  4.453  0.765  -0.177 -0.257
Hlth   3.815  -8.083 -2.289  -2.486 -1.936  0.693  0.765  7.820   3.496 -1.727
Utils  4.272  -9.617 -1.901   4.454 -2.343  2.721 -0.177  3.496  12.267 -4.055
Other -1.769   4.386  0.359  -3.865 -1.404 -1.272 -0.257 -1.727  -4.055  4.503

In [51]:
Std_Deviation = np.sqrt(np.diagonal(Covmatrix).round(2))
print ("Std_Deviation of the given data is :\n {} ".format(Std_Deviation))

Std_Deviation of the given data is :
 [2.33238076 5.16042634 1.7175564  4.38976081 2.25831796 2.16333077
 2.10950231 2.79642629 3.50285598 2.12132034] 


In [52]:
Expected__Mean_Vertical_axis  = pd.DataFrame( np.arange(0,0.1,0.005))
print (" Mean_Vertical_axis : \n {} ".format(Expected__Mean_Vertical_axis))

 Mean_Vertical_axis : 
         0
0   0.000
1   0.005
2   0.010
3   0.015
4   0.020
5   0.025
6   0.030
7   0.035
8   0.040
9   0.045
10  0.050
11  0.055
12  0.060
13  0.065
14  0.070
15  0.075
16  0.080
17  0.085
18  0.090
19  0.095 


In [53]:
def Weight_Lagrange_Multipliers(Expected_return):
    
    def Lambda_Cov_R(Expected_return):

        Lambda = np.divide( np.subtract( Delta.dot(Expected_return) , alpha ), np.subtract(zeta.dot(Delta),np.square(alpha)))
        Lambda_Cov_R_value  = np.multiply(Lambda,Covmatrix_inverse).dot(R_array)
        return (Lambda_Cov_R_value)

    def Second_Cov_R(Expected_return):
        y= np.divide(np.subtract(zeta,np.multiply(alpha,(0.02))),np.subtract (zeta.dot(Delta),np.square(alpha)))
        Second_Cov_R_value = np.multiply(y,Covmatrix_inverse).dot(e)
        return (Second_Cov_R_value)
    
    def weight(Expected_return):      
        
        Optimal_weight = pd.DataFrame(np.add(Lambda_Cov_R(Expected_return),Second_Cov_R(Expected_return)))
        return (Optimal_weight)
     
   
    weight(Expected_return)
    
    return np.add(Lambda_Cov_R(Expected_return),Second_Cov_R(Expected_return))


def Value_a(zeta,Covmatrix_inverse,e,alpha,R_array,Delta):
    return np.divide( np.multiply(zeta,Covmatrix_inverse).dot(e) - ( np.multiply(alpha,Covmatrix_inverse).dot(R_array)) ,np.subtract(np.multiply(zeta,Delta),np.square(alpha)) )


def Value_b(zeta,Covmatrix_inverse,e,alpha,R_array,Delta):
    return np.divide( np.multiply(Delta,Covmatrix_inverse).dot(R_array) - ( np.multiply(alpha,Covmatrix_inverse).dot(e)),np.subtract(np.multiply(zeta,Delta),np.square(alpha)) )


def Optimal_Portfolio_weight_a_b(zeta,Covmatrix_inverse,e,alpha,R_array,Delta,Expected_return):
    
    return np.add(Value_a(zeta,Covmatrix_inverse,e,alpha,R_array,Delta),(Value_b(zeta,Covmatrix_inverse,e,alpha,R_array,Delta)).dot(Expected_return))


def Std_deviation_Portfolio_with_RiskfreeAsset(Expected_Portfolio_Return,Risk_Free_Asset):
    ##(2 - 0.10)/np.sqrt(zeta - 2*alpha*0.13+Delta*np.square(0.13))
    return float((Expected_Portfolio_Return - Risk_Free_Asset)/(np.sqrt(zeta - 2*alpha*Risk_Free_Asset+Delta*np.square(Risk_Free_Asset))).astype(float))

def Sharpe_Ratio(Expected_Portfolio_Return,Riskfree_rate):
    
    return np.sqrt(zeta - 2*alpha*Riskfree_rate + Delta*np.square(Riskfree_rate))

In [55]:
e = pd.DataFrame([1] * len( df_premium.columns))
e  = np.array(e)
print ( "e Matrix :  \n {}".format(e))

e Matrix :  
 [[1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]
 [1]]


In [56]:
alpha = (R_array.T).dot(Covmatrix_inverse).dot(np.array(e))

R_array_Tran = np.array(R).T
zeta = R_array_Tran.dot(Covmatrix_inverse).dot(R_array)

e_Tran = np.array(e).T
Delta = e_Tran.dot(Covmatrix_inverse).dot(np.array(e))
print ( "Value of  \n Alpha : {} \n and \n Zeta :  {} \n and \n Delta : {} ".format(alpha,zeta,Delta))
print ( "\n Global Minimum Variance of Portfolio  : {} ".format(alpha/Delta))

NameError: name 'R_array' is not defined